In [ ]:
# ==============================================================================
# SECTION 1: LIBRARY IMPORTS
# ==============================================================================
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import os # Import os for path joining
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    MaxPooling1D,
    Dropout,
    Dense,
    LayerNormalization,
    GlobalAveragePooling1D,
    SeparableConv1D 
)
from tensorflow.keras.metrics import Recall, Precision

# Ensure reproducibility
tf.random.set_seed(42)
np.random.seed(42)


# ==============================================================================
# SECTION 2: DATA LOADING AND PREPROCESSING (CRITICAL PATH FIXED)
# ==============================================================================

# 🛑 PATH FIX APPLIED: Using 'r' prefix for raw strings to ignore escape sequences 🛑
# We are now using ABSOLUTE paths, so PATH_TO_DATA_FOLDER is no longer needed.
base_file_names = [
    r'C:\Users\sreej\OneDrive\Desktop\Deep Hybrid Model\backend\dataset\singlehop-indoor-moteid1-data.txt',
    r'C:\Users\sreej\OneDrive\Desktop\Deep Hybrid Model\backend\dataset\singlehop-indoor-moteid2-data.txt',
    r'C:\Users\sreej\OneDrive\Desktop\Deep Hybrid Model\backend\dataset\singlehop-outdoor-moteid3-data.txt',
    r'C:\Users\sreej\OneDrive\Desktop\Deep Hybrid Model\backend\dataset\singlehop-outdoor-moteid4-data.txt'
]

# FIX: file_paths is now just the list of absolute paths
file_paths = base_file_names

COLUMNS = ['Reading_Num', 'Mote_ID', 'Humidity', 'Temperature', 'Label']
all_mote_data = []

def load_and_clean_mote_data(file_path, skip_rows):
    """
    Loads and cleans a single mote data file by skipping junk rows 
    and manually assigning the known columns.
    """
    try:
        # 1. Load data
        # sep=r'\s+' handles both tabs and multiple spaces as delimiters
        df = pd.read_csv(
            file_path, 
            sep=r'\s+',              
            header=None,             
            skiprows=skip_rows,      
            engine='python',         
            usecols=range(5),        
            on_bad_lines='skip',
            encoding='utf-8' 
        )
        
        # 2. MANUALLY ASSIGN THE CORRECT COLUMN NAMES (Ensures 'Label' exists)
        df.columns = COLUMNS
        
        # 3. Robust type conversion and cleaning
        for col in ['Reading_Num', 'Mote_ID', 'Label']:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        
        for col in ['Humidity', 'Temperature']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

        # 4. Drop rows where critical data is missing
        df.dropna(subset=['Mote_ID', 'Humidity', 'Temperature', 'Label'], inplace=True)
        
        print(f"Loaded {len(df)} rows from {os.path.basename(file_path)}")
        return df.astype({'Mote_ID': int, 'Label': int}) 
    
    except FileNotFoundError:
        print(f"Error processing {file_path}: [Errno 2] File Not Found. Double-check your absolute C: path.")
        return pd.DataFrame()
    except Exception as e:
        # This catches residual issues like column reading errors, which the manual column assignment should fix.
        print(f"Error processing {file_path}. Data not included: {e}")
        return pd.DataFrame()

# Mote-specific loading using the required skiprows for this dataset:
skip_rules = {
    'singlehop-indoor-moteid1-data.txt': 4,
    'singlehop-indoor-moteid2-data.txt': 1,
    'singlehop-outdoor-moteid3-data.txt': 1,
    'singlehop-outdoor-moteid4-data.txt': 2
}

# Use the full path for loading, but the basename for the skip_rules lookup
for full_path in file_paths:
    # Use the filename (basename) to find the correct skip rule
    file_name_only = os.path.basename(full_path)
    all_mote_data.append(load_and_clean_mote_data(full_path, skip_rules.get(file_name_only, 0))) # Default to 0 if key not found

# Concatenate all DataFrames
merged_df = pd.concat(all_mote_data, ignore_index=True)

print(f"\nTotal merged data points: {len(merged_df)}")
if len(merged_df) > 0:
    print(f"Fault distribution:\n{merged_df['Label'].value_counts(normalize=True).mul(100).round(2)}")


# ==============================================================================
# SECTION 3: MODEL DEFINITION (EFFICIENT-TransCNN / TransCNN-Lite)
# (REMAINS UNCHANGED)
# ==============================================================================

if len(merged_df) > 0:
    # --- Sequence Creation ---
    TIME_STEP = 50  # The sequence length (or look-back window)
    FEATURES = ['Humidity', 'Temperature']
    N_FEATURES = len(FEATURES)

    X = merged_df[FEATURES].values
    y = merged_df['Label'].values

    # Standardize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    def create_sequences(X, y, time_steps):
        Xs, ys = [], []
        for i in range(len(X) - time_steps):
            Xs.append(X[i:(i + time_steps)])
            ys.append(y[i + time_steps]) 
        return np.array(Xs), np.array(ys)

    X_seq, y_seq = create_sequences(X_scaled, y, TIME_STEP)
    print(f"\nSequence input shape (N_samples, Time_Steps, N_Features): {X_seq.shape}")

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq
    )
    print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")


    def build_efficient_transcnn(input_shape, d_model=64, num_heads=4):
        inputs = Input(shape=input_shape)
        
        # 1. Efficient CNN Block
        x = SeparableConv1D(filters=d_model, kernel_size=5, padding='same', activation='relu', depth_multiplier=1, name='Depthwise_CNN_Block')(inputs)
        x = MaxPooling1D(pool_size=2)(x)
        x = Dropout(0.2)(x)
        
        # 2. Transformer/Attention Block
        attn_output = tf.keras.layers.MultiHeadAttention(key_dim=d_model, num_heads=num_heads, dropout=0.1, name='MHA_Block')(x, x)
        x = x + attn_output
        x = LayerNormalization(epsilon=1e-6)(x)
        
        # Feed Forward Network
        ffn_output = Dense(d_model * 2, activation="relu")(x)
        ffn_output = Dense(d_model)(ffn_output)
        ffn_output = Dropout(0.1)(ffn_output)
        
        x = x + ffn_output
        x = LayerNormalization(epsilon=1e-6)(x)
        
        # 3. Classifier Block
        z = GlobalAveragePooling1D()(x) 
        z = Dropout(0.5)(z)
        
        outputs = Dense(1, activation='sigmoid')(z)

        model = Model(inputs=inputs, outputs=outputs, name='Efficient_TransCNN')
        return model

    # Initialize and Compile
    model = build_efficient_transcnn(
        input_shape=(TIME_STEP, N_FEATURES), 
        d_model=64,
        num_heads=4
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy', 
        metrics=[ 'accuracy', Precision(name='precision'), Recall(name='recall') ]
    )

    print("\n--- Efficient-TransCNN Model Summary ---")
    model.summary()


    # ==============================================================================
    # SECTION 4: MODEL TRAINING AND EVALUATION
    # (REMAINS UNCHANGED)
    # ==============================================================================

    print("\n--- Starting Model Training ---")

    # Training Parameters
    EPOCHS = 50
    BATCH_SIZE = 64
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_split=0.1, 
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluation on Test Data
    print("\n--- Evaluating Model Performance ---")

    results = model.evaluate(X_test, y_test, verbose=0)
    loss, acc, precision, recall = results

    # Calculate F1-Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {acc:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test F1-Score: {f1_score:.4f}")

    # Classification Report and Confusion Matrix
    y_pred_proba = model.predict(X_test)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()

    print("\n--- Detailed Classification Report ---")
    print(classification_report(y_test, y_pred, target_names=['Normal (0)', 'Fault (1)']))

    print("\n--- Confusion Matrix ---")
    print(confusion_matrix(y_test, y_pred))

    model.save('efficient_transcnn_fault_diagnosis.h5')
    print("\nModel saved as 'efficient_transcnn_fault_diagnosis.h5'")
else:
    print("FATAL: Data loading failed. Please verify the absolute paths in 'base_file_names' list.")

In [ ]:
# ==============================================================================
# SECTION 6: VISUALIZATION (Plot Training History)
# ==============================================================================
import matplotlib.pyplot as plt # <--- THE FIX IS HERE
import numpy as np

# NOTE: This assumes 'history' variable from Section 4 is still in memory.
# If you closed your notebook, you must re-run the training section (Section 4) 
# to recreate the 'history' variable before running this cell!

try:
    plt.figure(figsize=(14, 6))

    # --- Plot 1: Loss ---
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss Over Epochs (Convergence)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (Binary Crossentropy)')
    plt.legend()
    plt.grid(True)

    # --- Plot 2: Accuracy/F1-Metrics ---
    
    # Custom function to calculate F1 from P & R in history
    # The array operations need to handle potential division by zero if P+R=0
    
    # Calculate Training F1-Score
    precision_train = np.array(history.history['precision'])
    recall_train = np.array(history.history['recall'])
    f1_train = np.divide(2 * precision_train * recall_train, 
                         precision_train + recall_train, 
                         out=np.zeros_like(precision_train), 
                         where=(precision_train + recall_train) != 0)

    # Calculate Validation F1-Score
    precision_val = np.array(history.history['val_precision'])
    recall_val = np.array(history.history['val_recall'])
    f1_val = np.divide(2 * precision_val * recall_val, 
                       precision_val + recall_val, 
                       out=np.zeros_like(precision_val), 
                       where=(precision_val + recall_val) != 0)


    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy', linestyle='--')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', linestyle='--')
    plt.plot(f1_train, label='Train F1-Score', linewidth=2)
    plt.plot(f1_val, label='Validation F1-Score', linewidth=2)
    
    plt.title('Model Performance Over Epochs (F1-Score is Key)')
    plt.xlabel('Epoch')
    plt.ylabel('Score')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

except NameError:
    print("FATAL ERROR: 'history' variable not found. You must run the training cell (Section 4) first!")
except Exception as e:
    print(f"An unexpected error occurred during plotting: {e}")

In [ ]:
# ==============================================================================
# SECTION 7: SINGLE-SAMPLE PREDICTION CHECK
# ==============================================================================
# Use one sample from the test set
sample_index = 0 
sample_X = X_test[sample_index].reshape(1, X_test.shape[1], X_test.shape[2])
sample_Y_true = y_test[sample_index]

# Predict the sample
prediction_proba = model.predict(sample_X)[0][0]
prediction_class = int(prediction_proba > 0.5)

print(f"--- Single Sample Prediction Check (Index {sample_index}) ---")
print(f"Actual Label (0=Normal, 1=Fault): {sample_Y_true}")
print(f"Predicted Probability of Fault (1): {prediction_proba:.4f}")
print(f"Predicted Class: {prediction_class}")

if prediction_class == sample_Y_true:
    print("Result: Correctly classified!")
else:
    print("Result: Misclassified. Analyze this sample for robustness.")

In [2]:
# ==============================================================================
# MODEL RECONSTRUCTION AND WEIGHT LOADING SCRIPT
# FIX: Adjusted final dense layer to Dense(1, activation='sigmoid') and loss
# to 'binary_crossentropy' to match the likely structure of the original saved model
# trained for binary classification.
# ==============================================================================
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv1D, DepthwiseConv1D, Activation, Add, GlobalAveragePooling1D, Dense, Dropout, Input
from tensorflow.keras.models import Model
import numpy as np

# --- 1. Define Custom Layers (Must be defined first) ---

class PositionalEmbedding(Layer):
    """
    Positional Embedding layer used in the Transformer part of the model.
    """
    def __init__(self, sequence_length, d_model, *args, **kwargs):
        if args and 'name' not in kwargs:
             kwargs['name'] = args[0]
        super().__init__(**kwargs) 
        self.sequence_length = sequence_length
        self.d_model = d_model
        self.position_embeddings = tf.keras.layers.Embedding(
            input_dim=sequence_length, output_dim=d_model
        )
        self.token_embeddings = tf.keras.layers.Conv1D(d_model, 1, padding="valid")

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        embedded_tokens = self.token_embeddings(inputs)
        return embedded_tokens + embedded_positions

    def get_config(self):
        config = super().get_config()
        config.update({"sequence_length": self.sequence_length, "d_model": self.d_model})
        return config


class TransformerBlock(Layer):
    """
    Standard Transformer Block composed of multi-head self-attention and a feed-forward network.
    """
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1, *args, **kwargs): 
        if args and 'name' not in kwargs:
             kwargs['name'] = args[0]
        super().__init__(**kwargs) 
        self.d_model = d_model
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(d_model)]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "d_model": self.d_model, "num_heads": self.num_heads, "ff_dim": self.ff_dim, "rate": self.rate,
        })
        return config

# --- 2. Define the Complete TransCNN Model Architecture ---
# NOTE: These parameters (sequence_length, num_features, etc.) MUST match 
# what was used when the model was originally trained. Please verify!

def build_transcnn_model(sequence_length=40, num_features=2, d_model=64, num_heads=4, ff_dim=128):
    """
    Builds the TransCNN hybrid model architecture.
    """
    inputs = Input(shape=(sequence_length, num_features))

    # --- CNN Feature Extraction Block ---
    x = Conv1D(filters=32, kernel_size=8, strides=1, activation='relu', padding='same')(inputs)
    x = Conv1D(filters=d_model, kernel_size=8, strides=1, activation='relu', padding='same')(x)
    
    # --- Transformer Block ---
    # Positional Embedding layer
    x = PositionalEmbedding(sequence_length=sequence_length, d_model=d_model)(x)
    
    # Transformer Block
    x = TransformerBlock(d_model=d_model, num_heads=num_heads, ff_dim=ff_dim)(x)

    # --- Classification Head ---
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x) 
    
    # FIX: Using 1 output neuron with sigmoid activation for binary classification
    outputs = Dense(1, activation='sigmoid')(x) 

    model = Model(inputs=inputs, outputs=outputs, name="TransCNN_Fault_Diagnosis")
    return model

# --- 3. Instantiate, Compile, and Load Weights ---

MODEL_FILE = 'efficient_transcnn_fault_diagnosis.h5'
NEW_MODEL_FILE = 'efficient_transcnn_fixed.h5'
# The final layer is now fixed to 1 neuron, so we don't need NUM_CLASSES explicitly in the rebuild function
SEQUENCE_LENGTH = 40 # Assuming your sequence length for input data
NUM_FEATURES = 2 # Assuming Humidity and Temperature (2 features)

try:
    # 3.1. Build the model architecture from scratch (Note: num_classes removed)
    new_model = build_transcnn_model(
        sequence_length=SEQUENCE_LENGTH, 
        num_features=NUM_FEATURES, 
    )
    
    # 3.2. Compile the model (MUST match original training setup)
    new_model.compile(
        optimizer='adam', 
        # FIX: Using binary_crossentropy for the 1-neuron sigmoid output layer
        loss='binary_crossentropy', 
        metrics=['accuracy']
    )
    
    print(f"Successfully reconstructed TransCNN model architecture.")

    # 3.3. Load ONLY the weights from the incompatible file
    new_model.load_weights(MODEL_FILE, by_name=True)
    print(f"Successfully loaded weights from {MODEL_FILE} into the new model structure.")

    # 3.4. Re-save the model in a fully compatible format
    new_model.save(NEW_MODEL_FILE)
    print(f"\nModel successfully saved to {NEW_MODEL_FILE}. It is now ready for TFLite conversion.")
    print("Next step: Re-run the 'Efficient-TransCNN TFLite Converter' Canvas.")

except Exception as e:
    print(f"CRITICAL FAILURE: Failed to reconstruct model or load weights. Error: {e}")
    print("If the error persists, there may be another layer mismatch. We may need to adjust the intermediate Dense(64) layer or the CNN block.")
    print("Please verify your original model building code if possible.")

Successfully reconstructed TransCNN model architecture.
Successfully loaded weights from efficient_transcnn_fault_diagnosis.h5 into the new model structure.

Model successfully saved to efficient_transcnn_fixed.h5. It is now ready for TFLite conversion.
Next step: Re-run the 'Efficient-TransCNN TFLite Converter' Canvas.


In [3]:
# ==============================================================================
# TFLITE CONVERSION SCRIPT
# Loads the fixed Keras model and converts it to a TensorFlow Lite model.
# ==============================================================================
import tensorflow as tf
import numpy as np
import os

# --- 1. Define Custom Layers (REQUIRED for Keras Model Loading) ---
# We must include the custom layer definitions again so Keras can correctly 
# load the architecture from the saved .h5 file.

class PositionalEmbedding(tf.keras.layers.Layer):
    """
    Positional Embedding layer used in the Transformer part of the model.
    """
    def __init__(self, sequence_length, d_model, *args, **kwargs):
        # Handle positional arguments in legacy Keras format
        if args and 'name' not in kwargs:
             kwargs['name'] = args[0]
        super().__init__(**kwargs) 
        self.sequence_length = sequence_length
        self.d_model = d_model
        self.position_embeddings = tf.keras.layers.Embedding(
            input_dim=sequence_length, output_dim=d_model
        )
        self.token_embeddings = tf.keras.layers.Conv1D(d_model, 1, padding="valid")

    def call(self, inputs):
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        embedded_tokens = self.token_embeddings(inputs)
        return embedded_tokens + embedded_positions

    def get_config(self):
        config = super().get_config()
        config.update({"sequence_length": self.sequence_length, "d_model": self.d_model})
        return config


class TransformerBlock(tf.keras.layers.Layer):
    """
    Standard Transformer Block composed of multi-head self-attention and a feed-forward network.
    """
    def __init__(self, d_model, num_heads, ff_dim, rate=0.1, *args, **kwargs): 
        # Handle positional arguments in legacy Keras format
        if args and 'name' not in kwargs:
             kwargs['name'] = args[0]
        super().__init__(**kwargs) 
        self.d_model = d_model
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=d_model)
        self.ffn = tf.keras.Sequential(
            [tf.keras.layers.Dense(ff_dim, activation="relu"), tf.keras.layers.Dense(d_model)]
        )
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "d_model": self.d_model, "num_heads": self.num_heads, "ff_dim": self.ff_dim, "rate": self.rate,
        })
        return config


# --- 2. Define Conversion Parameters ---

KERAS_MODEL_PATH = 'efficient_transcnn_fixed.h5'
TFLITE_MODEL_PATH = 'efficient_transcnn.tflite'
# The input shape of the model, determined by the original training script
# (Batch size, Sequence Length, Features)
INPUT_SHAPE = (1, 40, 2) 


# --- 3. Conversion Function ---

def representative_dataset_generator():
    """
    A generator function to provide representative data samples for 
    Post-Training Integer Quantization. 
    NOTE: Replace the dummy data generation below with actual preprocessed data
    from your dataset if you want to use full integer quantization.
    For now, we'll use dummy data for basic conversion.
    """
    # Using dummy data matching the expected input shape (40 steps, 2 features)
    for _ in range(10): # Generate 10 dummy samples
        yield [np.random.rand(*INPUT_SHAPE).astype(np.float32)]

def convert_to_tflite(keras_model_path, tflite_model_path):
    """
    Loads the Keras model and converts it to TFLite format.
    """
    if not os.path.exists(keras_model_path):
        print(f"ERROR: Keras model file not found at {keras_model_path}")
        return

    try:
        # Load the fixed Keras model, passing the custom objects
        custom_objects = {
            'PositionalEmbedding': PositionalEmbedding,
            'TransformerBlock': TransformerBlock
        }
        model = tf.keras.models.load_model(
            keras_model_path, 
            custom_objects=custom_objects
        )
        print(f"Successfully loaded Keras model from {keras_model_path}.")
    
    except Exception as e:
        print(f"CRITICAL ERROR: Failed to load the Keras model for conversion. Error: {e}")
        return

    # Initialize the TFLite converter
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    
    # --- Optimization Settings (Uncomment for desired optimization level) ---
    
    # 1. Default Optimization (Size reduction, generally safe)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

    # 2. Integer Quantization (For highest memory and computational saving)
    # This requires providing representative data.
    # converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # converter.representative_dataset = representative_dataset_generator
    # converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    # converter.inference_input_type = tf.int8  # Optional: depends on deployment
    # converter.inference_output_type = tf.int8 # Optional: depends on deployment

    try:
        # Perform the conversion
        tflite_model = converter.convert()
        
        # Save the TFLite model
        with open(tflite_model_path, 'wb') as f:
            f.write(tflite_model)
        
        print(f"\nSUCCESS: TFLite model saved to {tflite_model_path}")
        print(f"The size of the converted model is: {len(tflite_model) / 1024:.2f} KB.")
        
    except Exception as e:
        print(f"CRITICAL ERROR during TFLite conversion: {e}")
        print("This often happens due to incompatibility of custom layers with TFLite. Ensure all custom layer operations are TFLite-compatible.")

# --- 4. Execute Conversion ---
if __name__ == '__main__':
    convert_to_tflite(KERAS_MODEL_PATH, TFLITE_MODEL_PATH)


Successfully loaded Keras model from efficient_transcnn_fixed.h5.
INFO:tensorflow:Assets written to: C:\Users\sreej\AppData\Local\Temp\tmphn7ph7sw\assets


INFO:tensorflow:Assets written to: C:\Users\sreej\AppData\Local\Temp\tmphn7ph7sw\assets


Saved artifact at 'C:\Users\sreej\AppData\Local\Temp\tmphn7ph7sw'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 40, 2), dtype=tf.float32, name='input_layer_2')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2432776806736: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776807120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776808080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776807312: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776808656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776808272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776806928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776808848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776809040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776809424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2432776809